In [117]:
# Import libraries
import pandas as pd
import requests
from pprint import pprint

In [118]:
# Import Countries list from below API
url = 'https://restcountries.eu/rest/v2/all'

# Get response
response = requests.get(url).json()

# Process each country and store details in a list
cntry_ls = []
for index, each in enumerate(response):
    try:
        c_name = each['name']
        c_code = each['alpha3Code']
        c_lat = each['latlng'][0]
        c_lng = each['latlng'][1]
        c_pop = each['population']
        c_area = each['area']
        c_details = [c_name, c_code, c_pop, c_area, c_lat, c_lng]
        cntry_ls.append(c_details)
    except:
        print(f"Data not found for {c_name}")

Data not found for United States Minor Outlying Islands


In [119]:
# Convert list to dataframe        
cntry_df = pd.DataFrame(cntry_ls, columns=["Name", "Code", "Population", "Area", "Lat", "Lng"])

# Convert data to numeric
cntry_df["Population"] = pd.to_numeric(cntry_df["Population"])
cntry_df["Area"] = pd.to_numeric(cntry_df["Area"])
cntry_df["Lat"] = pd.to_numeric(cntry_df["Lat"])
cntry_df["Lng"] = pd.to_numeric(cntry_df["Lng"])

# Cross Join year
years = []
for i in range(2000, 2016, 1):
    years.append(i)
years_df = pd.DataFrame(years, columns=["Year"])
years_df

cntry_df['dummykey'] = 0
years_df['dummykey'] = 0

countries_df = cntry_df.merge(years_df, how='outer')

countries_df = countries_df.drop(columns=["dummykey"])
countries_df.head()

,Name,Code,Population,Area,Lat,Lng,Year
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004


In [121]:
# Get Emission data
emm_df = pd.read_csv("EmissionData.csv", skiprows=4)

# Remove unwanted columns
emm_df = emm_df.drop(columns=["Indicator Name", "Indicator Code"])
emm_df = emm_df.drop(columns=["1960", "1961", "1962", "1963", "1964", "1965", "1966", "1967", "1968", "1969"])
emm_df = emm_df.drop(columns=["1970", "1971", "1972", "1973", "1974", "1975", "1976", "1977", "1978", "1979"])
emm_df = emm_df.drop(columns=["1980", "1981", "1982", "1983", "1984", "1985", "1986", "1987", "1988", "1989"])
emm_df = emm_df.drop(columns=["1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999"])
emm_df = emm_df.drop(columns=["2016", "2017", "2018", "Unnamed: 63"])
emm_df.head()

,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,26.194875,25.934024,25.671162,26.420452,26.517293,27.200708,26.948260,27.895574,26.230847,25.915833,24.670529,24.505835,13.155542,8.351294,8.408363,NaN
1,Afghanistan,AFG,0.038506,0.039002,0.048716,0.051830,0.039378,0.052948,0.063728,0.085418,0.154101,0.241723,0.293837,0.412017,0.350371,0.315602,0.299445,NaN
2,Angola,AGO,0.580353,0.573047,0.720769,0.497975,0.996165,0.979740,1.098884,1.197844,1.181527,1.232495,1.243406,1.252789,1.330843,1.254617,1.291328,NaN
3,Albania,ALB,0.978175,1.053304,1.229541,1.412697,1.376213,1.412498,1.302576,1.322335,1.484311,1.495600,1.578574,1.803715,1.692908,1.749211,1.978763,NaN
4,Andorra,AND,8.019284,7.786950,7.590615,7.315761,7.358625,7.299872,6.746219,6.519466,6.427887,6.121652,6.122595,5.867130,5.916597,5.900753,5.832170,NaN


In [122]:
# Unpivot: Change year columns to rows
emm_df = pd.melt(emm_df, id_vars=['Country Name'], value_vars=['2000', '2001', '2002', '2003', '2004', '2005',
                                                       '2006', '2007', '2008', '2009', '2010', '2011',
                                                       '2012', '2013', '2014', '2015'])
emm_df.columns=["Name", "Year", "Emission"]
emm_df = emm_df.dropna()
emm_df["Year"] = pd.to_numeric(emm_df["Year"])
emm_df.head()

,Name,Year,Emission
0,Aruba,2000,26.194875
1,Afghanistan,2000,0.038506
2,Angola,2000,0.580353
3,Albania,2000,0.978175
4,Andorra,2000,8.019284


In [123]:
# Create results with countries and emission data
result = final_df.merge(emm_df, how='left', on=['Name','Year'])
result.head()

,Name,Code,Population,Area,Lat,Lng,Year,Emission
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000,0.038506
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001,0.039002
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002,0.048716
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003,0.051830
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004,0.039378
